# How to add human-in-the-loop processes to a ReAct agent (Functional API)

!!! info "Prerequisites"
    This guide assumes familiarity with the following:

    - Implementing [human-in-the-loop](../../concepts/human_in_the_loop) workflows with [interrupt](../../concepts/human_in_the_loop/#interrupt)
    - [How to create a ReAct agent using the Functional API](react-agent-from-scratch-functional.ipynb)

This guide demonstrates how to implement human-in-the-loop workflows in a ReAct agent using the LangGraph [Functional API](../../concepts/functional_api).

We will build off of the agent created in the [How to create a ReAct agent using the Functional API](react-agent-from-scratch-functional.ipynb) guide.

Specifically, we will demonstrate:

1. [How to review tool calls](#review-tool-calls-before-execution);
2. How to allow the model to reach out to a human for assistance.

Both of these goals can be accomplished through use of the [interrupt](../../concepts/human_in_the_loop/#interrupt) function at key points in our application.

## Setup

First, let's install the required packages and set our API keys:

In [1]:
%%capture --no-stderr
%pip install -U langgraph langchain-openai

In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("OPENAI_API_KEY")

<div class="admonition tip">
     <p class="admonition-title">Set up <a href="https://smith.langchain.com">LangSmith</a> for better debugging</p>
     <p style="padding-top: 5px;">
         Sign up for LangSmith to quickly spot issues and improve the performance of your LangGraph projects. LangSmith lets you use trace data to debug, test, and monitor your LLM aps built with LangGraph — read more about how to get started in the <a href="https://docs.smith.langchain.com">docs</a>. 
     </p>
 </div>

### Define model and tools

Let's first define the tools and model we will use for our example. As in the [ReAct agent guide](react-agent-from-scratch-functional.ipynb), we will use a single place-holder tool that gets a description of the weather for a location.

We will use an [OpenAI](https://python.langchain.com/docs/integrations/providers/openai/) chat model for this example, but any model [supporting tool-calling](https://python.langchain.com/docs/integrations/chat/) will suffice.

In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool

model = ChatOpenAI(model="gpt-4o-mini")


@tool
def get_weather(location: str):
    """Call to get the weather from a specific location."""
    # This is a placeholder for the actual implementation
    if any([city in location.lower() for city in ["sf", "san francisco"]]):
        return "It's sunny!"
    elif "boston" in location.lower():
        return "It's rainy!"
    else:
        return f"I am not sure what the weather is in {location}"


tools = [get_weather]

### Define tasks

We start with the same tasks from the [ReAct agent guide](react-agent-from-scratch-functional.ipynb):

1. **Call model**: We want to query our chat model with a list of messages.
2. **Call tool**: If our model generates tool calls, we want to execute them.

In [2]:
from langchain_core.messages import ToolMessage
from langgraph.func import entrypoint, task

tools_by_name = {tool.name: tool for tool in tools}


@task
def call_model(messages):
    """Call model with a sequence of messages."""
    response = model.bind_tools(tools).invoke(messages)
    return response


@task
def call_tool(tool_call):
    tool = tools_by_name[tool_call["name"]]
    observation = tool.invoke(tool_call["args"])
    return ToolMessage(content=observation, tool_call_id=tool_call["id"])


# @task
# def call_tool(proposed_tool_call):
#     review = review_tool_call(proposed_tool_call)
#     if isinstance(review, ToolMessage):
#         return review
#     else:
#         tool_call = review
#         tool = tools_by_name[tool_call["name"]]
#         observation = tool.invoke(tool_call["args"])
#         return ToolMessage(content=observation, tool_call_id=tool_call["id"])

## Review tool calls before execution

To review tool calls before execution, we add a call to [interrupt](../../concepts/human_in_the_loop/#interrupt) to our [entrypoint](../../concepts/functional_api/#entrypoint) implementation. When this function is called, execution will be paused until we issue a command to resume it.

The results of prior tasks-- in this case the initial model call-- are persisted, so that they are not run again following the `interrupt`.

Aside from this addition, our entrypoint implementation is the same as in the [ReAct agent guide](react-agent-from-scratch-functional.ipynb):

In [3]:
from typing import Union
from langchain_core.messages import ToolCall, ToolMessage


@task
def review_tool_call(tool_call: ToolCall) -> Union[ToolCall, ToolMessage]:
    """Review a tool call, returning a validated version."""
    human_review = interrupt(
        {
            "question": "Is this correct?",
            "tool_call": tool_call,
        }
    )
    review_action = human_review["action"]
    review_data = human_review.get("data")
    if review_action == "continue":
        return tool_call
    elif review_action == "update":
        updated_tool_call = {**tool_call, **{"args": review_data}}
        return updated_tool_call
    elif review_action == "feedback":
        return ToolMessage(
            content=review_data, name=tool_call["name"], tool_call_id=tool_call["id"]
        )

In [4]:
from langgraph.checkpoint.memory import MemorySaver
from langgraph.graph.message import add_messages
from langgraph.types import Command, interrupt


checkpointer = MemorySaver()

@entrypoint(checkpointer=checkpointer)
def agent(messages, previous):

    if previous is not None:
        messages = add_messages(previous, messages)

    llm_response = call_model(messages).result()
    while True:
        if not llm_response.tool_calls:
            break

        # Review tool calls
        tool_results = []
        tool_calls = []
        for tool_call in llm_response.tool_calls:
            review = review_tool_call(tool_call).result()
            if isinstance(review, ToolMessage):
                tool_results.append(review)
            else:
                tool_calls.append(review)

        # Execute remaining tool calls
        tool_result_futures = [call_tool(tool_call) for tool_call in tool_calls]
        remaining_tool_results = [fut.result() for fut in tool_result_futures]

        # Append to message list
        messages = add_messages(
            messages,
            [llm_response, *tool_results, *remaining_tool_results],
        )

        # Call model again
        llm_response = call_model(messages).result()

    # Generate final response
    messages = add_messages(messages, llm_response)
    return messages

## Usage

To use our agent, we invoke it with a messages list. Based on our implementation, these can be LangChain [message](https://python.langchain.com/docs/concepts/messages/) objects or OpenAI-style dicts:

In [8]:
def _print_step(step: dict) -> None:
    for task_name, result in step.items():
        if task_name in ("agent", "__interrupt__"):
            continue  # Just print task updates
        else:
            print(f"\n{task_name}:")
            if task_name == "review_tool_call":
                print(result)
            else:
                result.pretty_print()

### Accept a tool call

In [6]:
config = {"configurable": {"thread_id": "1"}}

In [7]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

for step in agent.stream([user_message], config):
    _print_step(step)

{'role': 'user', 'content': "What's the weather in san francisco?"}

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_P89CnI8LgJYwoEo5KAjoxDef)
 Call ID: call_P89CnI8LgJYwoEo5KAjoxDef
  Args:
    location: San Francisco


In [9]:
# highlight-next-line
human_input = Command(resume={"action": "continue"})

for step in agent.stream(human_input, config):
    _print_step(step)


review_tool_call:
{'name': 'get_weather', 'args': {'location': 'San Francisco'}, 'id': 'call_P89CnI8LgJYwoEo5KAjoxDef', 'type': 'tool_call'}

call_tool:
================================= Tool Message =================================

It's sunny!

call_model:
================================== Ai Message ==================================

The weather in San Francisco is sunny!


### Revise a tool call

In [15]:
config = {"configurable": {"thread_id": "2"}}

In [10]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

for step in agent.stream([user_message], config):
    _print_step(step)

{'role': 'user', 'content': "What's the weather in san francisco?"}

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_k4LihRKjdhBneMQPzfyOfK6G)
 Call ID: call_k4LihRKjdhBneMQPzfyOfK6G
  Args:
    location: San Francisco


In [11]:
# highlight-next-line
human_input = Command(resume={"action": "update", "data": {"location": "boston"}})

for step in agent.stream(human_input, config):
    _print_step(step)


review_tool_call:
{'name': 'get_weather', 'args': {'location': 'boston'}, 'id': 'call_k4LihRKjdhBneMQPzfyOfK6G', 'type': 'tool_call'}

call_tool:
================================= Tool Message =================================

It's rainy!

call_model:
================================== Ai Message ==================================

The weather in San Francisco is rainy!


### Generate a custom ToolMessage

In [9]:
config = {"configurable": {"thread_id": "3"}}

In [12]:
user_message = {"role": "user", "content": "What's the weather in san francisco?"}
print(user_message)

for step in agent.stream([user_message], config):
    _print_step(step)

{'role': 'user', 'content': "What's the weather in san francisco?"}

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_Qimc3WebYD8MIh8NWf47I4EM)
 Call ID: call_Qimc3WebYD8MIh8NWf47I4EM
  Args:
    location: San Francisco


In [13]:
# highlight-next-line
human_input = Command(
    # highlight-next-line
    resume={
        # highlight-next-line
        "action": "feedback",
        # highlight-next-line
        "data": "Please format as <City>, <State>.",
    # highlight-next-line
    },
# highlight-next-line
)

for step in agent.stream(human_input, config):
    _print_step(step)


review_tool_call:
content='Please format as <City>, <State>.' name='get_weather' id='49403506-0862-45ff-a8a8-9d94a8564155' tool_call_id='call_Qimc3WebYD8MIh8NWf47I4EM'

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_X03tBLOYU4P7sQ0xKci4dAv7)
 Call ID: call_X03tBLOYU4P7sQ0xKci4dAv7
  Args:
    location: San Francisco, CA

review_tool_call:
content='Please format as <City>, <State>.' name='get_weather' tool_call_id='call_X03tBLOYU4P7sQ0xKci4dAv7'

call_model:
================================== Ai Message ==================================
Tool Calls:
  get_weather (call_Pt4j9FfxqwXf6S9NNdJ88oxN)
 Call ID: call_Pt4j9FfxqwXf6S9NNdJ88oxN
  Args:
    location: San Francisco, California

review_tool_call:
content='Please format as <City>, <State>.' name='get_weather' tool_call_id='call_Pt4j9FfxqwXf6S9NNdJ88oxN'


Exception ignored in: <function WeakKeyDictionary.__init__.<locals>.remove at 0x1054ae200>
Traceback (most recent call last):
  File "/Users/chestercurme/.pyenv/versions/3.10.4/lib/python3.10/weakref.py", line 370, in remove
    def remove(k, selfref=ref(self)):
KeyboardInterrupt: 


KeyboardInterrupt: 

In [ ]:
# highlight-next-line
human_input = Command(resume={"action": "continue"})

for step in agent.stream(human_input, config):
    _print_step(step)